In [ ]:
# Create Stash app client

import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
from dataclasses import dataclass


@dataclass
class TagConversion:
    ai_tag: any
    target_tag: any

tag_conversion = {
    "Anal Fucking_AI": "Anal",
    "Ass Penetration_AI": "Anal",
    "Anal Gape_AI": "Anal Gape",
    "Ball Licking/Sucking_AI": "Ball Licking",
    "Blowjob_AI": "Blowjob",
    "Cumshot_AI": "Cumshot",
    "Cum on Person_AI": "Cumshot",
    "Deepthroat_AI": "Deepthroat",
    "Fingering_AI": "Fingering",
    "Grabbing Ass_AI": "Ass Grabbing",
    "Grabbing Boobs_AI": "Breast Squeezing",
    "Grabbing Hair/Head_AI": "Grabbing Hair/Head",
    "Handjob_AI": "Handjob",
    "Kissing_AI": "Kissing",
    "Licking Penis_AI": "Dick Licking",
    "Masturbation_AI": "Masturbation",
    "Pissing_AI": "Peeing",
    "Pussy Licking_AI": "Pussy Licking",
    "Pussy Licking (Clearly Visible)_AI": "Pussy Licking",
    "Pussy Rubbing_AI": "Pussy Rubbing",
    "Sucking Toy/Dildo_AI": "Toy Sucking",
    "Undressing_AI": "Striptease",
    "Vaginal Fucking_AI": "Vaginal Sex",
    "Vaginal Penetration_AI": "Vaginal Penetration",
    "Wet (Genitals)_AI": "Wet Genitals"
}

stash_tag_conversion = {}

for ai_tag_name in tag_conversion.keys():
    target_tag_name = tag_conversion[ai_tag_name]

    ai_tag = stash.find_tag(ai_tag_name)
    if not ai_tag:
        raise Exception("Could not find tag: " + ai_tag_name)
    
    target_tag = stash.find_tag(target_tag_name)
    if not target_tag:
        raise Exception("Could not find tag: " + tag_conversion[ai_tag_name])

    stash_tag_conversion[ai_tag["id"]] = TagConversion(ai_tag, target_tag)


In [ ]:
ai_reviewed_tag = stash.find_tag("AI_Reviewed")
ai_reviewed_tag_id = ai_reviewed_tag["id"]

In [ ]:
scene_id = 18562

markers = stash.get_scene_markers(scene_id)
ai_markers = [m for m in markers if m["title"].endswith("_AI")]

In [ ]:
for ai_marker in ai_markers:
    marker_ai_tag_id = ai_marker["primary_tag"]["id"]

    if marker_ai_tag_id not in stash_tag_conversion:
        print("Could find conversion for tag: " + ai_marker["primary_tag"]["name"])
        continue

    marker_target_tag = stash_tag_conversion[marker_ai_tag_id].target_tag
    if not marker_target_tag:
        print("Could not find target tag for: " + ai_marker["primary_tag"]["name"])
        continue

    update_marker = stash.update_scene_marker({
        "id": ai_marker["id"],
        "scene_id": ai_marker["scene"]["id"],
        "title": marker_target_tag["name"],
        "primary_tag_id": marker_target_tag["id"],
        "tag_ids": []
    })

scene = stash.find_scene(scene_id)
scene_tags = scene["tags"]
print("Existing tags:")
print([f"{t["id"]}: {t["name"]}" for t in scene_tags])

ai_marker_tags = [m["primary_tag"]["id"] for m in ai_markers]
ai_tags = [t for t in scene_tags if t["id"] in ai_marker_tags]
non_ai_tags = [t for t in scene_tags if t["id"] not in ai_marker_tags]

print("AI tags:")
print([f"{t["id"]}: {t["name"]}" for t in ai_tags])

print("Non AI tags:")
print([f"{t["id"]}: {t["name"]}" for t in non_ai_tags])

non_ai_tags_ids = [t["id"] for t in non_ai_tags]

update_scene = stash.update_scene({
    "id": scene_id,
    "tag_ids": non_ai_tags_ids
})

In [ ]:
scene = stash.find_scene(scene_id)
scene_tags = scene["tags"]
scene_tag_ids = [t["id"] for t in scene_tags] + [ai_reviewed_tag_id]

update_scene = stash.update_scene({
    "id": scene_id,
    "tag_ids": scene_tag_ids
})